In [ ]:
from pymongo import MongoClient
from datetime import datetime

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 12, 8, 0)},
        {"user_id": 1, "action": "view", "timestamp": datetime(2023, 4, 12, 9, 0)},
        {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)},
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()

**[문제 1: 특정 장르의 책 찾기]**

- **문제 설명**:
데이터베이스에 새로운 필드로 **`genre`**를 책 데이터에 추가하였습니다. 사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
"fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [80]:
from pymongo import MongoClient
import time

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용
    collection = db.books

    collection.delete_many({}) # 매번 지우고 다시 하고 귀찮으니깐 처음 부터 삭제

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009},
        {"title": "The Name of the Wind","author": "Patrick Rothfuss","year": 2007},
        {"title": "To Kill a Mockingbird", "author": "Harper Lee", "year": 1960, "genre": "Fiction"},
        {"title": "The Mistborn Trilogy", "author": "Brandon Sanderson", "year": 2006, "genre": "Fantasy"},
        {"title": "Pride and Prejudice", "author": "Jane Austen", "year": 1813, "genre": "Romance"},
        {"title": "The Catcher in the Rye", "author": "J.D. Salinger", "year": 1951, "genre": "Coming-of-age Fiction"}
    ]
    collection.insert_many(books)
    doc_count = collection.count_documents({})
    print(f'저장된 책의 개수는 {doc_count}개 입니다.')
    time.sleep(1)
    print("*"*8,f"찾으시는 책을 검색 중입니다.","*"*8) #시간이 나면 input도 해봐야지

    # for doc in collection.find({}, {"_id":0}): #이게 왜 반복이 되는거야.. 삭제 해야해...
    #     print(f"보관된 책을 불러 옴니다. {doc}", "\n") # 출력이 너무 지저분한거 같아
    return collection                              # 저장된 책의 개수만 출력이 되게 했다.

# 기존에 있던 데이터에 필드 genre를 추가하고 각각의 데이터에 genre를 추가한다
def set_genre(collection):         
    genre = {
        "Kafka on the Shore": "Magical Realism",
        "Norwegian Wood": "Contemporary Fiction",
        "1Q84": "Science Fiction",
        "The Name of the Wind":"Fantasy"
        }

    for title, genre in genre.items(): #.items으로 genre를 key,value으로 나누고 title로
        collection.update_many( # 각 데이터를 조회해서 값이 맞으면 genre를 추가한다.
            {"title":title},
            {"$set": {"genre":genre}}
        )

    # 특정 장르의 책 찾기    원래 식은 db.books.find({"genre":"fantasy"})를 함수로 표현해야 하는데
    #db.books.find({ "genre":"fantast"},{title: 1,author: 1 }) 이거도 해야돼...
def find_book(collection, genre):
    f_book = collection.find({"genre":genre},{"_id":0, "title":1, "author":1})

    # 여기부터
    if f_book:
        print(f"\n찾으시는 {genre}책의 제목과 저자 입니다.")
        for book in f_book:
            print(f'제목: {book["title"]}, 저자: {book["author"]}')
    else:
        print(f"{genre} 장르의 책을 찾을 수 없습니다.")
    #여기까지는 gpt...그래도 출력이 마음에 든다


collection = insert_data()
set_genre(collection)
time.sleep(2) #검색중 이라 쓰고 2초 뒤에 출력이 되게 만들었다. 이게 더 있어 보인다.
find_book(collection,"Fantasy")

저장된 책의 개수는 8개 입니다.
******** 찾으시는 책을 검색 중입니다. ********

찾으시는 Fantasy책의 제목과 저자 입니다.
제목: The Name of the Wind, 저자: Patrick Rothfuss
제목: The Mistborn Trilogy, 저자: Brandon Sanderson


**[문제 2: 감독별 평균 영화 평점 계산]**

- **문제 설명**:
각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**: 
모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[문제 3: 특정 사용자의 최근 행동 조회]**

- **문제 설명**:
특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[문제 4: 출판 연도별 책의 수 계산]**

- **문제 설명** :
데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[문제 5: 특정 사용자의 행동 유형 업데이트]**

- **문제 설명**:
특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.